# Get data

In [2]:
import warnings 
import numpy as np

warnings.filterwarnings("ignore")

## Retrieve the data from S3

In [3]:
from common_tools.aws.s3 import S3Client
import pandas as pd 
s3_client = S3Client()
data_path = s3_client.download_file('seloger-data-developers-dev','alaborde/data/data_query_20200220.csv')
data = pd.read_csv(data_path)

## Retrieve the ref_geo from S3

In [ ]:
from common_tools.aws.s3 import S3Client
import pandas as pd
s3_client = S3Client()
ref_geo_path = s3_client.download_file('seloger-data-developers-dev','alaborde/data/ref_geo.csv')
ref_geo = pd.read_csv(ref_geo_path)

# Prepare data

In [ ]:
data_prep = data[['idannonce','cp','ville','codeinsee','descriptif_fr',
             'px', 'surface','annee_construct','nb_pieces', 'latitude', 'longitude']].copy()

## Ads localization

### Geo mapper CP --> City

In [ ]:
key_city = ['city_name','city_id','city_code_insee_sl']
geomap_cp = ref_geo.drop_duplicates(subset=key_city).dropna(subset=key_city)[key_city]
geomap_cp = geomap_cp.rename(columns={'city_code_insee_sl': 'codeinsee'})

In [ ]:
data_prep_code_insee = data_prep.merge(geomap_cp, how='left',on=['codeinsee'])
data_prep_code_insee = data_prep_code_insee.rename(columns={
    "city_name": "city_name_by_code_insee",
    "city_id": 'city_id_by_code_insee'
})

### Project in Polygon

In [ ]:
import sys
sys.path.append('/home/jeanchristophe/codeFolder/sl_data-science-models/pricemap/src')
sys.path.append('/home/jeanchristophe/codeFolder/sl_data-science-models/priceml/src')
from pricemap.localization.looker import PolygonLookup
from pricemap.localization.utils import looker_dict, build_geo_mapper
from pricemap.filter.filters import lat_long_filter

iris_dict, _, _ = looker_dict(ref_geo)
iris_lookup = PolygonLookup(iris_dict)

In [ ]:
def localize(df: pd.DataFrame,
             looker_iris: PolygonLookup,
             ref_geo: pd.DataFrame) -> pd.DataFrame:
    df_to_localize = df.copy()
    df_to_localize.index.name = 'index'
    geo_map_iris, _, geo_map_city = build_geo_mapper(ref_geo)
    # Get dataframe with lat/lon information
    df_with_latlon = df_to_localize.dropna(subset=['latitude', 'longitude']).copy()

    # Look at the IRIS level
    iris_id_from_latlon = looker_iris.lookup(df_with_latlon[['latitude', 'longitude']])
    # Get projection table iris_id => dataframe index
    index_to_id_iris = iris_id_from_latlon.to_frame().dropna()
    index_to_id_iris.columns = ['iris_id']
    index_to_id_iris.index.name = 'index'
    # Merge dataframe with district id
    dataframe_with_idiris = df_to_localize.merge(index_to_id_iris, how='right', on='index')
    # And add the geo info
    geo_map_iris['iris_id'] = geo_map_iris['iris_id'].astype(int)
    dataframe_with_idiris = dataframe_with_idiris.merge(geo_map_iris, how='left', on='iris_id')
    return dataframe_with_idiris

In [ ]:
data_prep_code_insee_filt = lat_long_filter(data_prep_code_insee, 'latitude', 'longitude', False)
data_loc = localize(data_prep_code_insee_filt, iris_lookup, ref_geo)

In [ ]:
geomap_cp_bis = geomap_cp.rename(columns={"codeinsee": "city_code_insee_sl",
                                         'city_id': 'city_id_from_latlon'})
data_loc = data_loc.merge(geomap_cp_bis, how='left', on=['city_code_insee_sl'])

In [ ]:
data_localized = pd.concat([data_loc, data_prep_code_insee_filt], sort=False).drop_duplicates(subset=['idannonce'], keep='first')

In [ ]:
districts_mapper = ref_geo.dropna(subset=['districts_id','districts_name']).drop_duplicates(subset=['districts_id','districts_name'])
districts_mapper = districts_mapper[['districts_id','districts_name']].copy()

In [ ]:
data_localized_keep = data_localized[(data_localized['city_id_by_code_insee'] ==data_localized['city_id_from_latlon'])
              | (data_localized['city_id_from_latlon'].isna())]
data_localized_keep = data_localized_keep.merge(districts_mapper, how='left', on=['districts_id'])
data_localized_keep = data_localized_keep.drop(columns=['city_name', 'city_id_from_latlon','iris_id',
                                                       'department_id','region_id','country_id'])
data_loc = data_localized_keep.rename(columns={
    "city_name_by_code_insee": 'city_name',
    'city_id_by_code_insee': 'city_id',
    
}).copy()

In [ ]:
data_loc

## Save it locally

In [ ]:
import pickle
with open('/home/jeanchristophe/dataFolder/ppty_goldset/request_ads_2019_2020_iris_localized.pickle','wb') as handle:
    pickle.dump(data_loc, handle)

# Load it locally

In [ ]:
import pickle
with open('/home/jeanchristophe/dataFolder/ppty_goldset/request_ads_2019_2020_iris_localized.pickle','rb') as handle:
    data_loc = pickle.load(handle)

In [ ]:
data_loc.columns

## Focus on Montpellier area

In [ ]:
# Get department id
ref_geo[ref_geo.department_name=='Hérault'].dropna(subset=['department_id']).drop_duplicates('department_name')\
[['department_id','department_name']]

In [ ]:
# Get bordering cities
list_id_bordering_montpellier = eval(ref_geo[ref_geo.city_name=='Montpellier'].dropna(subset=['city_id']).drop_duplicates('city_id')\
['city_bordering'].values[0])
list_id_montpellier = list_id_bordering_montpellier + list(ref_geo[ref_geo.city_name=='Montpellier'].dropna(subset=['city_id']).drop_duplicates('city_id')\
[['city_id']].values[0])
list_id_montpellier

In [ ]:
# Filter in Montpellier
df_montpellier = data_loc[data_loc.city_id.isin(list_id_montpellier)].copy()
f'{df_montpellier.shape[0]} ads for Montpellier area'

## Focus on Grenoble area

In [ ]:
# Get bordering cities
list_id_bordering_grenoble= eval(ref_geo[ref_geo.city_name=='Grenoble'].dropna(subset=['city_id']).drop_duplicates('city_id')\
['city_bordering'].values[0])
list_id_grenoble = list_id_bordering_grenoble + list(ref_geo[ref_geo.city_name=='Grenoble'].dropna(subset=['city_id']).drop_duplicates('city_id')\
[['city_id']].values[0])
list_id_grenoble

In [ ]:
# Filter in Grenoble
df_grenoble = data_loc[data_loc.city_id.isin(list_id_grenoble)].copy()
f'{df_grenoble.shape[0]} ads for Grenoble area'

## Focus on Paris

In [ ]:
# Get bordering cities
list_id_bordering_paris18 = eval(ref_geo[ref_geo.city_name=='Paris 18ème'].dropna(subset=['city_id']).drop_duplicates('city_id')\
['city_bordering'].values[0])
list_id_paris18 = list_id_bordering_paris18 + list(ref_geo[ref_geo.city_name=='Paris 18ème'].dropna(subset=['city_id']).drop_duplicates('city_id')\
[['city_id']].values[0])
list_id_paris18

In [ ]:
# Filter in Paris 18eme
df_paris18 = data_loc[data_loc.city_id.isin(list_id_paris18)].copy()
f'{df_paris18.shape[0]} ads for Paris 18eme area'

# Gold set

In [ ]:
goldset = pd.concat([df_paris18, df_grenoble, df_montpellier], sort=False)

## Export goldset

In [ ]:
goldset.to_csv('/home/jeanchristophe/dataFolder/ppty_goldset/dedup_goldset.csv', index=False)

# Text preprocessing 

In [ ]:
import pandas as pd
goldset = pd.read_csv('/home/jeanchristophe/dataFolder/ppty_goldset/dedup_goldset.csv')

In [ ]:
from pricemap.filter.filters import simple_text
df_goldset_simpletxt = simple_text(goldset, 'descriptif_fr')

In [ ]:
data_goldset = df_goldset_simpletxt.copy()

# Find similar ads

In [ ]:
# Group ads that are around the three targeted area
# Method: project in ref geo to get the department and group the ads

group_mapper= ['city_id','region_id']
depart_mapper = ref_geo.drop_duplicates(subset=group_mapper).dropna(subset=group_mapper)[group_mapper]

In [ ]:
data_goldset_depart_code = data_goldset.merge(depart_mapper, how='left', on=['city_id'])
data_goldset_depart_code.region_id.value_counts()                      

In [ ]:
def group_function(df, delta_price, delta_surface, delta_piece):
    # Delta price of 5%
    # Create max and min bound for each price
    df['min_px'] = df.px*(1-(delta_price/100))
    df['max_px'] = df.px*(1+(delta_price/100))
    
    # Create max and min bound for each price
    df['min_surface'] = df.surface*(1-(delta_surface/100))
    df['max_surface'] = df.surface*(1+(delta_surface/100))

    df['min_nbpiece'] = df.nb_pieces-delta_nbpiece
    df['max_nbpiece'] = df.nb_pieces+delta_nbpiece
    
    piece_matrix= np.zeros((df.shape[0], df.shape[0]))
    surface_matrix= np.zeros((df.shape[0], df.shape[0]))
    price_matrix= np.zeros((df.shape[0], df.shape[0]))
    
    for i in range(df.shape[0]):
        id_groups = np.where((df.nb_pieces >= df.iloc[i]['min_nbpiece']) & (df.nb_pieces <= df.iloc[i]['max_nbpiece']))
        piece_matrix[i, id_groups] = 1
        id_groups = np.where((df.surface >= df.iloc[i]['min_surface']) & (df.surface <= df.iloc[i]['max_surface']))
        surface_matrix[i, id_groups] = 1
        id_groups = np.where((df.px >= df.iloc[i]['min_px']) & (df.px <= df.iloc[i]['max_px']))
        price_matrix[i, id_groups] = 1
    
    combined_matrix = np.multiply(np.multiply(piece_matrix, surface_matrix), price_matrix)
    df['groups'] = 'nan'
    df['len_group'] = 0
    # Get groups
    for i in range(combined_matrix.shape[0]):
        index_list = df.index[np.where(combined_matrix[i,:])[0].tolist()].tolist()
        get_index = df.index[i]
        if get_index in index_list:
            index_list.remove(get_index)
        id_annoncegroup = df.loc[index_list]['idannonce'].values.tolist()
        df.at[get_index, 'groups'] = str(id_annoncegroup)
        df.at[get_index, 'len_group'] = len(id_annoncegroup)
    return df



In [ ]:
df_montpellier = data_goldset_depart_code[data_goldset_depart_code.region_id == 240].copy()
df_paris18 = data_goldset_depart_code[data_goldset_depart_code.region_id == 239].copy()
df_grenoble= data_goldset_depart_code[data_goldset_depart_code.region_id == 252].copy()

### Paris

In [ ]:
def get_len(row):
    return len(eval(row['groups']))

In [ ]:
delta_price = 3.5
delta_surface = 3
delta_nbpiece = 1

groupped_paris18 = group_function(df_paris18, delta_price, delta_surface, delta_nbpiece)


In [ ]:
groupped_paris18.apply(get_len, axis = 1).hist()

In [ ]:
print(np.sum(groupped_paris18.apply(get_len, axis = 1) == 0), 'idannonce in paris18 has 0 propositions')
print(np.mean(groupped_paris18.apply(get_len, axis = 1)), 'mean number of propositions in paris18')
print(max(groupped_paris18.apply(get_len, axis = 1)), 'max number of proposition in paris18')

In [ ]:
groupped_paris18.head()

### Montpellier

In [ ]:
delta_price = 3.5
delta_surface = 3
delta_nbpiece = 1

groupped_montpellier = group_function(df_montpellier, delta_price, delta_surface, delta_nbpiece)

In [53]:
print(np.sum(groupped_montpellier.apply(get_len, axis = 1) == 0), 'idannonce in Montpellier has 0 propositions')
print(np.mean(groupped_montpellier.apply(get_len, axis = 1)), 'mean number of propositions in Montpellier')
print(max(groupped_montpellier.apply(get_len, axis = 1)), 'max number of proposition in Montpellier')

228 idannonce in montpellier has 0 propositions
5.92672858617131 mean number of propositions in montpellier
32 max number of proposition in montpellier


In [54]:
groupped_montpellier.head()

,idannonce,cp,ville,codeinsee,descriptif_fr,px,surface,annee_construct,nb_pieces,latitude,...,districts_name,region_id,min_px,max_px,min_surface,max_surface,min_nbpiece,max_nbpiece,groups,len_group
1850,153882117,34000,Montpellier,340172.0,pres d arenes t de m a deux pas du centre hist...,196000.0,49.0,NaN,2.0,43.604727,...,Port Marianne,240,189140.00,202860.00,47.53,50.47,1.0,3.0,"[154505991, 152613925, 145473413, 154043351, 1...",9
1851,137167811,34000,Montpellier,340172.0,au premier etage d une residence securisee t d...,138112.0,46.0,NaN,2.0,43.604727,...,Port Marianne,240,133278.08,142945.92,44.62,47.38,1.0,3.0,"[146351053, 149451371, 154220755]",3
1852,153356669,34000,Montpellier,340172.0,herault a vendre a montpellier dans le secteur...,229000.0,106.0,1970.0,4.0,43.604727,...,Port Marianne,240,220985.00,237015.00,102.82,109.18,3.0,5.0,"[153217207, 151328671]",2
1853,154506007,34000,Montpellier,340172.0,appartement d exception dans residence haut de...,650000.0,93.0,NaN,4.0,43.604727,...,Port Marianne,240,627250.00,672750.00,90.21,95.79,3.0,5.0,"[153207129, 146798901, 151359923, 142369085, 1...",5
1854,153597569,34430,Saint Jean de Vedas,340270.0,st jean de vedas proche de l a de la station d...,399000.0,110.0,2018.0,4.0,43.568759,...,Sud,240,385035.00,412965.00,106.70,113.30,3.0,5.0,[],0


### Grenoble

In [ ]:
delta_price = 3.5
delta_surface = 3
delta_nbpiece = 1

groupped_grenoble = group_function(df_grenoble, delta_price, delta_surface, delta_nbpiece)

In [58]:
print(np.sum(groupped_grenoble.apply(get_len, axis = 1) == 0), 'idannonce in Grenoble has 0 propositions')
print(np.mean(groupped_grenoble.apply(get_len, axis = 1)), 'mean number of propositions in Grenoble')
print(max(groupped_grenoble.apply(get_len, axis = 1)), 'max number of proposition in Grenoble')

173 idannonce in Grenoble has 0 propositions
3.2607709750566896 mean number of propositions in Grenoble
18 max number of proposition in Grenoble


In [59]:
groupped_grenoble.head()

,idannonce,cp,ville,codeinsee,descriptif_fr,px,surface,annee_construct,nb_pieces,latitude,...,districts_name,region_id,min_px,max_px,min_surface,max_surface,min_nbpiece,max_nbpiece,groups,len_group
968,152917265,38100,Grenoble,380185.0,grenoble allee de la pelouse en bordure du par...,88000.0,80.08,NaN,4.0,45.162621,...,Arlequin,252,84920.00,91080.00,77.6776,82.4824,3.0,5.0,"[152715315, 137918345, 152656465, 154105923]",4
969,151210099,38100,Grenoble,380185.0,era immobilier gresivaudan a trouve pour vous ...,158970.0,70.00,1960.0,3.0,45.180660,...,Allies-Clos d'Or,252,153406.05,164533.95,67.9000,72.1000,2.0,4.0,"[150604827, 147897763, 142827435, 145299099, 1...",5
970,150907823,38130,Échirolles,380151.0,iad france sophie armand vous propose echiroll...,125000.0,63.00,1970.0,4.0,45.147226,...,Comboire,252,120625.00,129375.00,61.1100,64.8900,3.0,5.0,"[151785547, 152595879, 154223735, 152288085]",4
971,148550245,38000,Grenoble,380185.0,appartement residence etudiante marie curie de...,67000.0,18.00,NaN,1.0,45.194349,...,Berriat-Ampere,252,64655.00,69345.00,17.4600,18.5400,0.0,2.0,[],0
972,152658935,38400,Saint Martin d Heres,380421.0,secteur parc jo blanchon au calme et proche tr...,199000.0,99.00,NaN,6.0,45.173628,...,Renaudie-La Plaine,252,192035.00,205965.00,96.0300,101.9700,5.0,7.0,[151928147],1


### Checks if idannonce is unique

In [67]:
print(groupped_paris18.set_index('idannonce').index.is_unique)
print(groupped_montpellier.set_index('idannonce').index.is_unique)
print(groupped_grenoble.set_index('idannonce').index.is_unique)

True
True
True


### Saving

In [70]:
def get_group_list(df, min_len, max_len):
    df_filtered = df[(df.len_group >= min_len) & (df.len_group <= max_len)].copy()
    list_group = []
    for _, row in df_filtered.iterrows():
        group = [row.idannonce] + eval(row.groups)
        list_group.append(group)
    return list_group
    

In [76]:
# Groups on Paris
list_of_groups = get_group_list(groupped_paris18, 4, 20)
list_idannonce_flat = [item for sublist in list_of_groups for item in sublist]
list_ids_paris = list(set(list_idannonce_flat))
f'{len(list_ids_paris)} idannonce in Paris'

'224 idannonce in Paris'

In [77]:
# Groups on Motnpellier
list_of_groups = get_group_list(groupped_montpellier, 4, 20)
list_idannonce_flat = [item for sublist in list_of_groups for item in sublist]
list_ids_montpellier = list(set(list_idannonce_flat))
f'{len(list_ids_montpellier)} idannonce in Montpellier'

'1396 idannonce in Montpellier'

In [78]:
# Groups on Motnpellier
list_of_groups = get_group_list(groupped_grenoble, 4, 20)
list_idannonce_flat = [item for sublist in list_of_groups for item in sublist]
list_ids_grenoble = list(set(list_idannonce_flat))
f'{len(list_ids_grenoble)} idannonce in Grenoble'

'433 idannonce in Grenoble'

In [80]:
list_ids_annotation = list_ids_paris + list_ids_montpellier + list_ids_grenoble
len(list_ids_annotation)

2053

In [81]:
import json
with open('/home/jeanchristophe/dataFolder/ppty_goldset/idannonce_dedup.json', 'w') as handle:
    json.dump(list_ids_annotation, handle)

In [82]:
groups_goldataset = pd.concat([groupped_grenoble, groupped_montpellier, groupped_paris18], sort=False)

In [83]:
groupped_paris18.len_group.describe()

count    968.000000
mean       1.616736
std        2.065326
min        0.000000
25%        0.000000
50%        1.000000
75%        2.000000
max       11.000000
Name: len_group, dtype: float64

In [84]:
groups_goldataset.len_group.describe()

count    3788.000000
mean        4.204593
std         4.616089
min         0.000000
25%         1.000000
50%         3.000000
75%         6.000000
max        32.000000
Name: len_group, dtype: float64

In [86]:
groups_goldataset.to_csv('/home/jeanchristophe/dataFolder/ppty_goldset/goldset_dedup_grouped.csv', index=False)

# Check image access

In [87]:
import json
with open('/home/jeanchristophe/dataFolder/ppty_goldset/idannonce_dedup.json', 'r') as handle:
    selected_id_annonce = json.load(handle)

In [90]:
import pandas as pd
id_photo = pd.read_excel('/home/jeanchristophe/codeFolder/ppty_goldset/data/mapping_id_tag.xlsx')

In [91]:
id_photo

,idvisuel,idannonce,dt,ordre,width,height,idtypevisuel,url,fichier,thumb_url,...,si_plan,codif_plan,si_censure_descriptif,idTypeSource,idEtatVisuel,si_censure,b600_width,b600_height,tagphoto,tagfile
0,828278883,126755075,2019-01-22 21:21:00,0,300.0,225.0,1,NaN,NaN,NaN,...,0.0,NaN,0,2,1,0,NaN,NaN,0lsge31naiqxuipdpbu7i0j1x02t2nz4qe2kwwxnc,NaN
1,828278885,126755075,2019-01-22 21:21:00,1,300.0,225.0,1,NaN,NaN,NaN,...,0.0,NaN,0,2,1,0,NaN,NaN,0c2l4rd5hhvxn9jtiigaoot7ivh8wc69cagglgxg8,NaN
2,828278887,126755075,2019-01-22 21:21:00,2,300.0,225.0,1,NaN,NaN,NaN,...,0.0,NaN,0,2,1,0,NaN,NaN,0vrmyps5fyspy86fb00bsjaiadr2ge61q76bd2c0o,NaN
3,828278889,126755075,2019-01-22 21:21:00,3,300.0,225.0,1,NaN,NaN,NaN,...,0.0,NaN,0,2,1,0,NaN,NaN,0xtvuw00vzbonxbo57dfcxkgpszcn8la4l0n2njvc,NaN
4,828278891,126755075,2019-01-22 21:21:00,4,300.0,225.0,1,NaN,NaN,NaN,...,0.0,NaN,0,2,1,0,NaN,NaN,0zcana0tan6fkt3lznpraujersd3kcwk6sbqpme6w,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12605,967479613,154787717,2020-01-06 23:30:00,4,300.0,225.0,1,NaN,NaN,NaN,...,0.0,NaN,0,2,1,0,NaN,NaN,0a4nadtvcy5e5pfmwp14mejdyqhkkyz9g76kaxf5s,NaN
12606,965551425,154787717,2019-12-31 13:59:00,5,300.0,225.0,1,NaN,NaN,NaN,...,0.0,NaN,0,2,1,0,NaN,NaN,0dq28nnwgma7k4a03i8u8v28sv3n1lry3zc3brmww,NaN
12607,965551427,154787717,2019-12-31 13:59:00,6,300.0,225.0,1,NaN,NaN,NaN,...,0.0,NaN,0,2,1,0,NaN,NaN,23vllbd93239vdk9zc22avps9o3f23myh1dcjk3wc,NaN
12608,967479617,154787717,2020-01-06 23:30:00,7,300.0,225.0,1,NaN,NaN,NaN,...,0.0,NaN,0,2,1,0,NaN,NaN,0xe3vnnpwh40o3srff931xlv9vrekk5tt9ess4pa8,NaN
